## A description of the problem and a discussion of the background.

Mishelle's Company sent her on a job to Abuja, Nigeria and have reserved an accomodation for her at the Hilton Hotels and Resort. She is currently on a fittness challenge and would like to see it through. The problem is to locate a gym fairly close to Mishelle, and which has standard facilities.

## A description of how to solve the problem

First of all, we import the necessary libraries that we need to solve the solve the problem such as numpy, pamdas, follium, requests... We would be calling on foursquare APIs.
Next we use geocode to find the address the Hilton Hotels and Resorts on the map (i.e; its longlitude and latitude) after defining our foursquare credentials ofcourse. Then we search for nearby gyms, and recomend the gym with the most ratings to Mishelle, giving her its location.

##### Importing Libraries

In [19]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed
Libraries imported.


##### We now define our foursquare credentials

In [20]:
CLIENT_ID = 'SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC' # your Foursquare ID
CLIENT_SECRET = 'OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC
CLIENT_SECRET:OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1


##### Getting the Latitude and Longitude values of the Hilton Hotels and  Resorts

In [21]:
address = '1 AGUIYI IRONSI STREET MAITAMA, ABUJA, 900001, NIGERIA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

9.0819482 7.490408


##### passing a search query to foursquare for nearby gyms

In [25]:
search_query = 'Gyms'
radius = 5000
print(search_query + ' .... OK!')

Gyms .... OK!


##### Defining url

In [26]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC&client_secret=OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1&ll=9.0819482,7.490408&v=20180604&query=Gyms&radius=5000&limit=30'

##### Getting results

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0b84d14c1f671905c23372'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
       'suffix': '.png'},
      'id': '4f4528bc4b90abdf24c9de85',
      'name': 'Athletics & Sports',
      'pluralName': 'Athletics & Sports',
      'primary': True,
      'shortName': 'Athletics & Sports'}],
    'hasPerk': False,
    'id': '523b34b311d22a9a911d62cc',
    'location': {'address': 'Wuse 2',
     'cc': 'NG',
     'city': 'Abuja',
     'country': 'Nigeria',
     'distance': 1882,
     'formattedAddress': ['Wuse 2',
      'Abuja',
      'Federal Capital Territory',
      'Nigeria'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 9.078092691288381,
       'lng': 7.473729727941876}],
     'lat': 9.078092691288381,
     'lng': 7.473729727941876,
     'state': 'Federal Capital Territory'},
    'name': 'Gym Sigma Apartments',
    'referralId': 'v-1544258769'}]}}

#####  Transfering relevant part of  JSON file into a pandas dataframe 

In [28]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'shortName': 'Athletics & Sports', 'id': '4f...",False,523b34b311d22a9a911d62cc,Wuse 2,NG,Abuja,Nigeria,1882,"[Wuse 2, Abuja, Federal Capital Territory, Nig...","[{'lat': 9.078092691288381, 'label': 'display'...",9.078093,7.47373,Federal Capital Territory,Gym Sigma Apartments,v-1544258769


Unfortunately, there are not many gyms near Mishelle, but there is one at wuse which is fairly close to her.